In [7]:
# | default_exp backend.tokens

In [8]:
# | export
import tiktoken
from typing import Dict, List, Any, Optional


In [9]:
# | export
class TokenManager:
    def __init__(self, model_name: str = "gpt-4", max_tokens: int = 128000):
        self.model_name = model_name
        self.max_tokens = max_tokens
        self.compression_threshold = 0.8  # 80%
        try:
            self.encoding = tiktoken.encoding_for_model(model_name)
        except:
            self.encoding = tiktoken.get_encoding("cl100k_base")  # fallback
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in text"""
        return len(self.encoding.encode(text))
    
    def count_conversation_tokens(self, messages: List[Dict]) -> int:
        """Count total tokens in conversation history"""
        total = 0
        for msg in messages:
            content = msg.get("content", "")
            if isinstance(content, str):
                total += self.count_tokens(content)
            # Add overhead for message structure
            total += 10
        return total
    
    def should_compress(self, messages: List[Dict]) -> bool:
        """Check if compression is needed"""
        current_tokens = self.count_conversation_tokens(messages)
        return current_tokens >= (self.max_tokens * self.compression_threshold)
    
    def compress_history(self, messages: List[Dict]) -> List[Dict]:
        """Compress conversation history while preserving context"""
        if len(messages) <= 2:  # Keep system + at least 1 message
            return messages
        
        system_msg = messages[0] if messages[0]["role"] == "system" else None
        recent_messages = messages[-4:]  # Keep last 4 messages
        
        # Compress middle messages into summary
        middle_messages = messages[1:-4] if len(messages) > 5 else []
        
        compressed = []
        if system_msg:
            compressed.append(system_msg)
        
        if middle_messages:
            summary = self._create_summary(middle_messages)
            compressed.append({
                "role": "assistant",
                "content": f"[COMPRESSED HISTORY - {len(middle_messages)} messages summarized]\n{summary}"
            })
        
        compressed.extend(recent_messages)
        return compressed
    
    def _create_summary(self, messages: List[Dict]) -> str:
        """Create concise summary of conversation history"""
        topics = []
        actions = []
        
        for msg in messages:
            content = msg.get("content", "")
            role = msg.get("role", "")
            
            if role == "user":
                # Extract key topics/requests
                if any(word in content.lower() for word in ["create", "build", "make"]):
                    topics.append("Creation tasks")
                if any(word in content.lower() for word in ["fix", "debug", "error"]):
                    topics.append("Debugging")
                if any(word in content.lower() for word in ["analyze", "review", "check"]):
                    topics.append("Analysis")
                if any(word in content.lower() for word in ["deploy", "install", "setup"]):
                    topics.append("Deployment")
            
            elif role == "assistant":
                # Extract actions taken
                if "fs_read" in content:
                    actions.append("File reading")
                if "fs_write" in content:
                    actions.append("File editing")
                if "execute_bash" in content:
                    actions.append("Command execution")
        
        summary_parts = []
        if topics:
            summary_parts.append(f"Topics: {', '.join(set(topics))}")
        if actions:
            summary_parts.append(f"Actions: {', '.join(set(actions))}")
        
        return "; ".join(summary_parts) if summary_parts else "General conversation"
